In [77]:
import pandas as pd 
import numpy as np 

In [78]:
credits=pd.read_csv('tmdb_5000_credits.csv')
movies=pd.read_csv('tmdb_5000_movies.csv')
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [79]:
movies=movies.merge(credits,on='title')

In [80]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


# useful columns for project
# genres
# id
# keywords
# title
# overview
# cast	
# crew

In [81]:
movies=movies[['movie_id','genres','keywords','title','overview','cast','crew']]

In [82]:
# checking null values in dataset
movies.isna().sum()

movie_id    0
genres      0
keywords    0
title       0
overview    3
cast        0
crew        0
dtype: int64

In [83]:
# dropping null values 
movies.dropna(inplace=True)

In [84]:
# checking any duplicates in dataset
movies.duplicated().sum()

0

In [85]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [86]:
import ast

In [87]:
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [88]:
movies['genres']=movies['genres'].apply(convert)


In [89]:
movies['keywords']=movies['keywords'].apply(convert)


In [90]:
def convert3(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3:
            l.append(i['name'])
            counter+=1
        else:
            break
    return l


In [91]:
movies['cast']=movies['cast'].apply(convert3)

In [92]:
def fetch_director(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l


In [93]:
movies['crew']=movies['crew'].apply(fetch_director)

In [94]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [95]:
movies.head(1)

,movie_id,genres,keywords,title,overview,cast,crew
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [96]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(' ','') for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(' ','') for i in x])


In [97]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [98]:
movies.head(1)

,movie_id,genres,keywords,title,overview,cast,crew,tags
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [99]:
new_df=movies[['movie_id','title','tags']]


In [100]:
import warnings
warnings.filterwarnings('ignore')

In [101]:
new_df['tags']=new_df['tags'].apply(lambda x:' '.join(x))

In [102]:
import nltk

In [103]:
from nltk import PorterStemmer
ps=PorterStemmer()


In [104]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)


In [105]:
new_df['tags']=new_df['tags'].apply(stem)

In [106]:
from sklearn.metrics.pairwise import cosine_similarity

In [112]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [113]:
similarity=cosine_similarity(vector)

In [114]:
print(list(similarity[0]))

[1.0000000000000002, 0.08346223261119858, 0.08603090020146065, 0.0734718358370645, 0.1892994097121204, 0.10838874619051501, 0.04024218182927669, 0.14673479641335554, 0.05923488777590923, 0.0967301666813349, 0.10259783520851541, 0.09464970485606021, 0.09037128496931669, 0.04499212706658476, 0.12824729401064427, 0.06282808624375433, 0.07894736842105264, 0.13977653617040256, 0.09493290614465533, 0.0830812984794528, 0.058038100008800934, 0.10968169942141635, 0.0662266178532522, 0.08740748201220976, 0.0533380747062665, 0.05101627678885769, 0.15389675281277312, 0.18693292157876878, 0.116543309349613, 0.065033247714309, 0.06684847767323797, 0.15907119074394446, 0.08520286456846099, 0.09733285267845754, 0.0, 0.09933992677987831, 0.17316974359835272, 0.07894736842105264, 0.08111071056538127, 0.08226127456606226, 0.07694837640638656, 0.16563698349810535, 0.0, 0.09086217008485092, 0.03382550457458692, 0.08240856434303293, 0.1391037210186643, 0.19672236884115843, 0.08447772061910234, 0.05827165467

In [115]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(1214, 0.28676966733820225),
 (2405, 0.26901379342448517),
 (3728, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.25038669783359574)]

In [ ]:
def recommend (movie):
    movie_index = new_df [new_df['title'] == movie].index[0]
    distances = similarity [movie_index]
    movies_list = sorted (list(enumerate (distances)), reverse=True, key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Batman Begins')

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf


In [ ]:
new_df.iloc[3295]

movie_id                                                13197
title                                             10th & Wolf
tags        a former street tough return to hi philadelphi...
Name: 3296, dtype: object

In [ ]:
new_df[new_df['title']=='batman begins'].index[0]

119

In [ ]:
new_df.head()

,movie_id,title,tags
0,19995,avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,pirates of the caribbean: at world's end,"captain barbossa, long believ to be dead, ha c..."
2,206647,spectre,a cryptic messag from bond’ past send him on a...
3,49026,the dark knight rises,follow the death of district attorney harvey d...
4,49529,john carter,"john carter is a war-weary, former militari ca..."


In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())


In [ ]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
    

In [ ]:
# vector = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(new_df.to_dict(),open('movies.pkl','wb'))

In [116]:
pickle.dump(similarity,open('similarity.pkl','wb'))